<a href="https://colab.research.google.com/github/cstimmons/datawork/blob/main/Admission_prediction_with_pyspark_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 1 : Install Dependencies & Run a SparkSession


In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 198 kB 58.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=d8bb778ba8f904678b4e277854ddf532e3cd237b6b1625df8f054980b3a7fa4d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spark').getOrCreate()

# TASK 2 : Clone & Explore dataset

In [4]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [5]:
#check the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [7]:
#create a spark dataframe
df = spark.read.csv("/content/admission_dataset/Admission_Predict_Ver1.1.csv", header=True, inferSchema=True)


In [8]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [13]:
#get the no.of rows & columns
print(df.count(),len(df.columns))

500 9


In [14]:
#print schema 
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [15]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [18]:
#drop the unnecessary column
df = df.drop('Serial No')


In [19]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [20]:
#check for null values
for i in df.columns:
  print(i+":",df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [21]:
# correlation analysis
for col in df.columns:
  print('correlation to chance of admission for {} is {}'.format(col,df.stat.corr('Chance of Admit',col)))

correlation to chance of admission for GRE Score is 0.8103506354632598
correlation to chance of admission for TOEFL Score is 0.7922276143050823
correlation to chance of admission for University Rating is 0.6901323687886892
correlation to chance of admission for SOP is 0.6841365241316723
correlation to chance of admission for LOR is 0.6453645135280112
correlation to chance of admission for CGPA is 0.882412574904574
correlation to chance of admission for Research is 0.5458710294711379
correlation to chance of admission for Chance of Admit is 1.0


In [22]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols= ['GRE Score', 'TOEFL Score', 'CGPA'],outputCol='features')

In [24]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [26]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features', 'Chance of Admit')

In [27]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [28]:
#split the dataset into training and testing set
train, test = final_data.randomSplit([0.7,0.3])

In [29]:
#build & train the model
models = LinearRegression(featuresCol='features', labelCol='Chance of Admit')
model = models.fit(train)

In [32]:
#get coefficients & intercept
print('coefficients:',model.coefficients)
print('intercept:',model.intercept)

coefficients: [0.002176349408429626,0.004103976426315886,0.13583084822708397]
intercept: -1.5730325381462007


In [39]:
#get summary of the model
summary = model.summary

In [40]:
#print the rmse & r2 score
print('RMSE',summary.rootMeanSquaredError)
print('r2',summary.r2)

RMSE 0.060709001379881763
r2 0.8045570442827726


# TASK 6 : Evaluate & Save the Model

In [48]:
#transform on the test data
predictions = model.transform(test)

In [49]:
#display the predictions
predictions.show(20)

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|[290.0,104.0,7.46]|           0.45| 0.4982204664092891|
| [294.0,95.0,7.64]|           0.49|0.49443962888703985|
|  [295.0,93.0,7.2]|           0.46| 0.4286424522229211|
| [295.0,96.0,7.34]|           0.47| 0.4599707002536604|
|[296.0,101.0,7.68]|            0.6|  0.528849420190878|
| [297.0,96.0,7.89]|           0.43| 0.5390303655954158|
| [297.0,98.0,7.67]|           0.59| 0.5173555318380891|
|  [298.0,98.0,7.5]|           0.44| 0.4964406370479144|
|  [298.0,99.0,7.6]|           0.46| 0.5141276982969385|
|[298.0,100.0,7.95]|           0.58| 0.5657724716027339|
| [299.0,94.0,7.34]|           0.42|0.46046814503474676|
| [299.0,96.0,7.86]|           0.54| 0.5393081389654626|
|[299.0,100.0,7.42]|           0.42|0.49595847145080874|
|[299.0,100.0,7.89]|           0.59| 0.5597989701175383|
| [300.0,95.0,8.22]|           

In [51]:
#evaluate the model 
from pyspark.ml.evaluation import RegressionEvaluator
evalaluator = RegressionEvaluator(predictionCol= 'prediction',labelCol= 'Chance of Admit',metricName='r2')
print('r2 on the test data',evalaluator.evaluate(predictions))

r2 on the test data 0.8020846570107139


In [52]:
#save the model
model.save("model")

In [53]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('model')